Showing how the framework can be loaded:

In [1]:
from fiatmodel import Calibration

Testing the interface

In [ ]:
# defining MESH calibration parameter bounds
class_dict_bounds = {
    1: {
        'sno': [0.0, 0.5],
        'rhos': [0.0, 0.5],
        'clay1': [10, 50],
        'sand1': [30, 70],
        'clay3': [20, 30],
    },
    3: {
        'lamx': [0.0, 1.0],
        'cmas': [0.0, 1.0],
        'clay2': [20, 60],
        'clay1': [0, 10],
    },
}

hydrology_dict_bounds = {
    1: {
        'zsnl': [0, 10],
        'zplg': [5, 15]
    },
    12: {
        'zpls': [0, 42],
        'zsnl': [0, 10],
        'zplg': [5, 15]
    },
}

routing_dict_bounds = {
    1: {
        'r2n': [0.0, 1.0],
        'r1n': [0.0, 1.0]
    },
    3: {
        'pwr': [0.0, 1.0],
        'flz': [0.0, 1.0]
    },
}

In [ ]:
cal_experiment = Calibration(
    calibration_software = 'ostrich',
    model_software = 'mesh',
    calibration_config = {
        'random_seed': int(time.time()),
        'algorithm': 'DDS',
        'algorithm_specs': { # refer to software manual for the keys
            'PerturbationValue': 0.2,
            'MaxIteration': 10_000,
            'UseRandomParamValue': None,
        },
        'metric': 'kge_2012', # refer to HydrErr documentation for valid keys
    },
    model_config = {
        'instance_path': './wolf-creek-research-basin/mesh/',
        'calibration_bounds': {
            'class': class_dict_bounds,
            'hydrology': hydrology_dict_bounds,
            'routing': routing_dict_bounds,
        },
        'constraints': {
            'class': class_constraints,
            'hydrology': None,
            'routing': None,
        },
    }
)